In [1]:
dataset_name = "facebook_pages"
output_path = "/home/ofrie.r/graph-ncp-project/results/snap_with_whiskers+lgc/"+dataset_name+"/"

In [4]:
import pandas as pd

import pandas as pd

# -------- Load CSV files --------
lgc = pd.read_csv(
    "/home/ofrie.r/graph-ncp-project/results/lgc/facebook_pages/ncp_min_clusters_by_size.csv"
)
snap = pd.read_csv(
    "/home/ofrie.r/graph-ncp-project/results/snap_calc_with_whiskers/facebook_pages/ncp.musae_facebook_no_self_loops.clusters.tab",
    sep="\t"   # Tab-separated
)

# Strip any spaces in column names
lgc.columns = lgc.columns.str.strip()
snap.columns = snap.columns.str.strip()


# Rename columns to standard names
lgc = lgc.rename(columns={
    "size": "size",
    "conductance": "phi",
    "cluster_nodes": "nodes"
})
snap = snap.rename(columns={
    "#Size": "size",
    "Phi": "phi",
    "Nodes": "nodes"
})


# Convert nodes column to string
lgc["nodes"] = lgc["nodes"].astype(str)
snap["nodes"] = snap["nodes"].astype(str)



# -------- Merge both DataFrames on 'size' using an outer join --------
merged = pd.merge(
    lgc[["size", "phi", "nodes"]],
    snap[["size", "phi", "nodes"]],
    on="size",
    how="outer",
    suffixes=("_lgc", "_snap")
)

# -------- Optional: inspect merged result --------
print(merged.head())

# -------- Select minimum phi for each size --------
rows = []
for _, row in merged.iterrows():
    size = row["size"]
    
    phi_lgc  = row["phi_lgc"]
    phi_snap = row["phi_snap"]

    # handle missing values
    if pd.isna(phi_lgc):
        chosen_phi = phi_snap
        chosen_nodes = row["nodes_snap"]
        chosen_source = "snap"
    elif pd.isna(phi_snap):
        chosen_phi = phi_lgc
        chosen_nodes = row["nodes_lgc"]
        chosen_source = "lgc"
    else:
        # choose minimum phi
        if phi_lgc <= phi_snap:
            chosen_phi = phi_lgc
            chosen_nodes = row["nodes_lgc"]
            chosen_source = "lgc"
        else:
            chosen_phi = phi_snap
            chosen_nodes = row["nodes_snap"]
            chosen_source = "snap"

    rows.append({
        "size": size,
        "phi": chosen_phi,
        "nodes": chosen_nodes,
        "source": chosen_source
    })

result = pd.DataFrame(rows)

# -------- Save to new CSV --------
result.to_csv(output_path+"combined_min_phi.csv", index=False)

print("Done. Saved combined_min_phi.csv")


   size   phi_lgc                   nodes_lgc  phi_snap  \
0   1.0       NaN                         NaN  1.000000   
1   2.0       NaN                         NaN  0.333333   
2   3.0  0.142857           10121 19234 20840  0.142857   
3   4.0  0.076923       6902 15942 19313 9249  0.076923   
4   5.0  0.047619  15108 17489 7729 805 12828  0.047619   

                    nodes_snap  
0                       12600   
1                  12600 6056   
2            15497 21372 1972   
3       9249 6902 19313 15942   
4  17489 7729 805 15108 12828   
Done. Saved combined_min_phi.csv
